In [1]:
print("file can run!")

file can run!


In [2]:
import pandas as pd
#constants
token='^NSEBANK'
duration=60
test_len=300

In [3]:
#load data

df = pd.read_csv(f'{token}_{duration}.csv')
df=df.drop(['Volume'],axis=1)
df['Datetime']=pd.to_datetime(df['Datetime'])
df.head(10)

,Datetime,Open,High,Low,Close,Adj Close
0,2024-02-16 09:15:00+05:30,46454.300781,46461.601562,46354.898438,46381.851562,46381.851562
1,2024-02-16 09:20:00+05:30,46375.949219,46383.351562,46327.500000,46328.648438,46328.648438
2,2024-02-16 09:25:00+05:30,46323.800781,46417.500000,46319.500000,46397.550781,46397.550781
3,2024-02-16 09:30:00+05:30,46403.648438,46428.648438,46315.648438,46339.851562,46339.851562
4,2024-02-16 09:35:00+05:30,46347.699219,46385.101562,46331.800781,46380.250000,46380.250000
5,2024-02-16 09:40:00+05:30,46378.601562,46395.050781,46298.800781,46335.300781,46335.300781
6,2024-02-16 09:45:00+05:30,46333.750000,46333.750000,46268.351562,46278.300781,46278.300781
7,2024-02-16 09:50:00+05:30,46278.750000,46357.300781,46266.000000,46342.050781,46342.050781
8,2024-02-16 09:55:00+05:30,46342.449219,46385.500000,46339.000000,46377.500000,46377.500000
9,2024-02-16 10:00:00+05:30,46377.398438,46407.750000,46357.699219,46399.351562,46399.351562


In [4]:
df.describe()

,Open,High,Low,Close,Adj Close
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,47549.377835,47579.578089,47517.853121,47549.790066,47549.790066
std,853.360070,851.529382,854.120700,852.702060,852.702060
min,45706.449219,45758.000000,45665.148438,45718.398438,45718.398438
25%,46830.788086,46866.050781,46800.449219,46834.886719,46834.886719
50%,47516.125000,47543.125000,47490.351562,47520.175781,47520.175781
75%,48074.175781,48110.925781,48047.950195,48074.625000,48074.625000
max,49946.250000,49974.351562,49892.601562,49947.500000,49947.500000


In [5]:
#add day of  features
df['day_week']=df['Datetime'].dt.dayofweek
df['day_month']=df['Datetime'].dt.day
df['month']=df['Datetime'].dt.month


df['time']=df['Datetime'].map(lambda a:(pd.Timedelta(a-pd.to_datetime('2004-04-14 9:15:00+05:30'))/pd.Timedelta('5m'))%75)
df['open_lag_3']=df['Open'].shift(-3)
df['open_dif']=df["Open"]-df['open_lag_3']
df['dtime']=df.index
df['tar']=df['open_dif'].map(lambda a: 1 if a>0 else 0)
df=df.drop(['open_lag_3'],axis=1)



In [6]:
#onehotencoding for days of week

from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(df[['day_week']]))
OH_cols_train

,0,1,2,3,4
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
4419,0.0,0.0,0.0,1.0,0.0
4420,0.0,0.0,0.0,1.0,0.0
4421,0.0,0.0,0.0,1.0,0.0
4422,0.0,0.0,0.0,1.0,0.0


In [7]:
for i in OH_cols_train:
    df[f'week_day_{i}']=OH_cols_train[i]
[f'week_day_{i}' for i in range(5)]

['week_day_0', 'week_day_1', 'week_day_2', 'week_day_3', 'week_day_4']

In [8]:
df.tail(10)

,Datetime,Open,High,Low,Close,Adj Close,day_week,day_month,month,time,open_dif,dtime,tar,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4
4414,2024-05-16 14:40:00+05:30,47670.199219,47750.898438,47654.148438,47746.398438,47746.398438,3,16,5,71.0,-154.402344,4414,0,0.0,0.0,0.0,1.0,0.0
4415,2024-05-16 14:45:00+05:30,47744.601562,47787.550781,47704.851562,47757.351562,47757.351562,3,16,5,72.0,-132.546875,4415,0,0.0,0.0,0.0,1.0,0.0
4416,2024-05-16 14:50:00+05:30,47755.750000,47842.949219,47750.300781,47825.949219,47825.949219,3,16,5,73.0,-171.351562,4416,0,0.0,0.0,0.0,1.0,0.0
4417,2024-05-16 14:55:00+05:30,47824.601562,47877.949219,47769.351562,47873.800781,47873.800781,3,16,5,74.0,-123.699219,4417,0,0.0,0.0,0.0,1.0,0.0
4418,2024-05-16 15:00:00+05:30,47877.148438,47940.898438,47871.601562,47928.949219,47928.949219,3,16,5,0.0,-96.550781,4418,0,0.0,0.0,0.0,1.0,0.0
4419,2024-05-16 15:05:00+05:30,47927.101562,47998.949219,47925.550781,47949.000000,47949.000000,3,16,5,1.0,-97.199219,4419,0,0.0,0.0,0.0,1.0,0.0
4420,2024-05-16 15:10:00+05:30,47948.300781,47983.851562,47948.148438,47973.699219,47973.699219,3,16,5,2.0,-95.148438,4420,0,0.0,0.0,0.0,1.0,0.0
4421,2024-05-16 15:15:00+05:30,47973.699219,48052.898438,47962.398438,48022.800781,48022.800781,3,16,5,3.0,NaN,4421,0,0.0,0.0,0.0,1.0,0.0
4422,2024-05-16 15:20:00+05:30,48024.300781,48051.601562,48004.199219,48042.898438,48042.898438,3,16,5,4.0,NaN,4422,0,0.0,0.0,0.0,1.0,0.0
4423,2024-05-16 15:25:00+05:30,48043.449219,48046.500000,47958.050781,47977.398438,47977.398438,3,16,5,5.0,NaN,4423,0,0.0,0.0,0.0,1.0,0.0


In [9]:

def make_lag(label:str,lags:list):
    l=[]
    for i in lags:
        df[f'{label}_lag_{i}']=df[label].shift(i)
        l.append(f'{label}_lag_{i}')
    print(l)


def make_moving_avg(label:str,sizes:list):
    l=[]
    for i in sizes:
        df[f'{label}_moving_avg_{i}']=df[label].rolling(window=i, center=True,min_periods=i//2).mean() 
        l.append(f'{label}_moving_avg_{i}')
    print(l)

def make_fourier(period:int,label,order):
    from statsmodels.tsa.deterministic import Fourier
    global df
    fourier = Fourier(period=period*75, order=order)
    donkey=fourier.in_sample(df['Close'])
    l=[]
    for i in donkey:
        df.insert(5,f'{label}_{i}',list(donkey[i]),True)
        l.append(f'{label}_{i}')
    print(l)


def test_make_lag(label:str,lags:list):
    l=[]
    for i in lags:
        test_df[f'{label}_lag_{i}']=test_df[label].shift(i)
        l.append(f'{label}_lag_{i}')
    print(l)


def test_make_moving_avg(label:str,sizes:list):
    l=[]
    for i in sizes:
        test_df[f'{label}_moving_avg_{i}']=test_df[label].rolling(window=i, center=True,min_periods=i//2).mean() 
        l.append(f'{label}_moving_avg_{i}')
    print(l)

def test_make_fourier(period:int,label,order):
    from statsmodels.tsa.deterministic import Fourier
    global test_df
    fourier = Fourier(period=period*75, order=order)
    donkey=fourier.in_sample(test_df['Close'])
    l=[]
    for i in donkey:
        test_df.insert(5,f'{label}_{i}',list(donkey[i]),True)
        l.append(f'{label}_{i}')
    print(l)



import matplotlib.pyplot as plt
def cplot(labels:list,rows=df.shape[0]):
    df.tail(rows).plot(y=labels,x='dtime')
    plt.show()
    

# make_fourier(75,'Open',4)
# df

In [10]:

test_df=df.loc[list(range(len(df)-test_len,len(df))),:]
df=df.loc[list(range(len(df)-test_len)),:]


In [11]:
df

,Datetime,Open,High,Low,Close,Adj Close,day_week,day_month,month,time,open_dif,dtime,tar,week_day_0,week_day_1,week_day_2,week_day_3,week_day_4
0,2024-02-16 09:15:00+05:30,46454.300781,46461.601562,46354.898438,46381.851562,46381.851562,4,16,2,36.0,50.652344,0,1,0.0,0.0,0.0,0.0,1.0
1,2024-02-16 09:20:00+05:30,46375.949219,46383.351562,46327.500000,46328.648438,46328.648438,4,16,2,37.0,28.250000,1,1,0.0,0.0,0.0,0.0,1.0
2,2024-02-16 09:25:00+05:30,46323.800781,46417.500000,46319.500000,46397.550781,46397.550781,4,16,2,38.0,-54.800781,2,0,0.0,0.0,0.0,0.0,1.0
3,2024-02-16 09:30:00+05:30,46403.648438,46428.648438,46315.648438,46339.851562,46339.851562,4,16,2,39.0,69.898438,3,1,0.0,0.0,0.0,0.0,1.0
4,2024-02-16 09:35:00+05:30,46347.699219,46385.101562,46331.800781,46380.250000,46380.250000,4,16,2,40.0,68.949219,4,1,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,2024-05-10 15:05:00+05:30,47436.898438,47468.648438,47406.851562,47440.101562,47440.101562,4,10,5,73.0,-8.601562,4119,0,0.0,0.0,0.0,0.0,1.0
4120,2024-05-10 15:10:00+05:30,47440.101562,47453.398438,47392.000000,47429.300781,47429.300781,4,10,5,74.0,-0.148438,4120,0,0.0,0.0,0.0,0.0,1.0
4121,2024-05-10 15:15:00+05:30,47424.050781,47453.750000,47420.199219,47449.300781,47449.300781,4,10,5,0.0,34.250000,4121,1,0.0,0.0,0.0,0.0,1.0
4122,2024-05-10 15:20:00+05:30,47445.500000,47455.250000,47423.550781,47440.550781,47440.550781,4,10,5,1.0,54.199219,4122,1,0.0,0.0,0.0,0.0,1.0


In [12]:
labels=['Open','Close','tar','open_dif','High','Low']
lags=[1,3,6]
for i in labels:
    make_lag(i,lags)

make_moving_avg('Open',[1,5])
make_moving_avg('Close',[1,5])
make_moving_avg('High',[1,5])
make_moving_avg('Low',[1,5])


make_fourier(1,'Open',4)
make_fourier(1,'Close',4)

['Open_lag_1', 'Open_lag_3', 'Open_lag_6']
['Close_lag_1', 'Close_lag_3', 'Close_lag_6']
['tar_lag_1', 'tar_lag_3', 'tar_lag_6']
['open_dif_lag_1', 'open_dif_lag_3', 'open_dif_lag_6']
['High_lag_1', 'High_lag_3', 'High_lag_6']
['Low_lag_1', 'Low_lag_3', 'Low_lag_6']
['Open_moving_avg_1', 'Open_moving_avg_5']
['Close_moving_avg_1', 'Close_moving_avg_5']
['High_moving_avg_1', 'High_moving_avg_5']
['Low_moving_avg_1', 'Low_moving_avg_5']
['Open_sin(1,75)', 'Open_cos(1,75)', 'Open_sin(2,75)', 'Open_cos(2,75)', 'Open_sin(3,75)', 'Open_cos(3,75)', 'Open_sin(4,75)', 'Open_cos(4,75)']
['Close_sin(1,75)', 'Close_cos(1,75)', 'Close_sin(2,75)', 'Close_cos(2,75)', 'Close_sin(3,75)', 'Close_cos(3,75)', 'Close_sin(4,75)', 'Close_cos(4,75)']


In [13]:
labels=['Open','Close','tar','open_dif','High','Low']
lags=[1,3,6]
for i in labels:
    test_make_lag(i,lags)

test_make_moving_avg('Open',[1,5])
test_make_moving_avg('Close',[1,5])
test_make_moving_avg('High',[1,5])
test_make_moving_avg('Low',[1,5])


test_make_fourier(1,'Open',4)
test_make_fourier(1,'Close',4)

['Open_lag_1', 'Open_lag_3', 'Open_lag_6']
['Close_lag_1', 'Close_lag_3', 'Close_lag_6']
['tar_lag_1', 'tar_lag_3', 'tar_lag_6']
['open_dif_lag_1', 'open_dif_lag_3', 'open_dif_lag_6']
['High_lag_1', 'High_lag_3', 'High_lag_6']
['Low_lag_1', 'Low_lag_3', 'Low_lag_6']
['Open_moving_avg_1', 'Open_moving_avg_5']
['Close_moving_avg_1', 'Close_moving_avg_5']
['High_moving_avg_1', 'High_moving_avg_5']
['Low_moving_avg_1', 'Low_moving_avg_5']
['Open_sin(1,75)', 'Open_cos(1,75)', 'Open_sin(2,75)', 'Open_cos(2,75)', 'Open_sin(3,75)', 'Open_cos(3,75)', 'Open_sin(4,75)', 'Open_cos(4,75)']
['Close_sin(1,75)', 'Close_cos(1,75)', 'Close_sin(2,75)', 'Close_cos(2,75)', 'Close_sin(3,75)', 'Close_cos(3,75)', 'Close_sin(4,75)', 'Close_cos(4,75)']


In [14]:
df=df.dropna()
test_df=test_df.dropna()

In [15]:
'''
making a basic model to find how it works with basic labels 
'''
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
labels=['Close', 'Close_cos(1,75)', 'Close_cos(2,75)', 'Close_cos(3,75)', 'Close_cos(4,75)', 'Close_lag_1',
        'Close_lag_3', 'Close_lag_6', 'Close_moving_avg_1', 'Close_moving_avg_5', 'Close_sin(1,75)', 'Close_sin(2,75)',
        'Close_sin(3,75)', 'Close_sin(4,75)', 'High', 'High_lag_1', 'High_lag_3', 'High_lag_6', 'High_moving_avg_1', 
        'High_moving_avg_5', 'Low', 'Low_lag_1', 'Low_lag_3', 'Low_lag_6', 'Low_moving_avg_1', 'Low_moving_avg_5', 
        'Open', 'Open_cos(1,75)', 'Open_cos(2,75)', 'Open_cos(3,75)', 'Open_cos(4,75)', 'Open_lag_1', 'Open_lag_3', 
        'Open_lag_6', 'Open_moving_avg_1', 'Open_moving_avg_5', 'Open_sin(1,75)', 'Open_sin(2,75)', 'Open_sin(3,75)',
        'Open_sin(4,75)', 'day_month', 'dtime', 'month', 'open_dif_lag_1', 'open_dif_lag_3', 'open_dif_lag_6', 
        'tar_lag_1', 'tar_lag_3', 'tar_lag_6', 'time', 'week_day_0', 'week_day_1', 'week_day_2', 'week_day_3',
        'week_day_4']

print(len(labels))
X=df[labels]
y=df[['tar']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=300, random_state=0,shuffle=False)




55


In [16]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [17]:
y_refore = pd.Series(model.predict(X_train)[:,0], index=y_train.index)
y_fore = pd.Series(model.predict(X_test)[:,0], index=y_test.index)

In [18]:
def cerror(X,R):
    n=0
    for i in range(len(X)):

        if R[i]>0.5:
            m=1
        else:
            m=0
        if X[i]==m:
            n+= 1 
        
    print(n/len(X)*100)
cerror(list(y_train['tar']),list(y_refore))
cerror(list(y_test['tar']),list(y_fore))

85.38501833420639
89.66666666666666


In [19]:
from sklearn.metrics import mean_absolute_error
err=mean_absolute_error(y_test,y_fore)
err2=mean_absolute_error(y_train,y_refore)


print(err,err2)


0.287967173039724 0.3147842266932094


In [20]:
test_X=test_df[labels]
test_y=test_df['tar']

y_true_fore = pd.Series(model.predict(test_X)[:,0], index=test_y.index)

mean_absolute_error(y_true_fore,test_y)

0.3142436900116081

In [21]:
cerror(list(test_y),list(y_true_fore))

84.19243986254295


In [22]:
# x=pd.to_datetime('2024-05-14 9:15:00+05:30')

# s=pd.to_datetime('2024-05-14 9:15:00+05:30')
# pd.Timedelta(x-s)/pd.Timedelta('5m')